In [ ]:
import numpy as np
from IPython.display import display, HTML
from pydrake.examples.manipulation_station import ManipulationStation

import matplotlib.pyplot as plt, mpld3

import pydrake
from pydrake.all import (
    DiagramBuilder, ConnectMeshcatVisualizer, Simulator, FindResourceOrThrow,
    Parser, MultibodyPlant, RigidTransform, LeafSystem, BasicVector,
    JacobianWrtVariable, RollPitchYaw, SignalLogger, AddTriad,
    PiecewisePolynomial, PiecewiseQuaternionSlerp, RotationMatrix, Solve,
    TrajectorySource, BodyIndex
)
from pydrake.multibody import inverse_kinematics
from pydrake.all import SnoptSolver, IpoptSolver
from pydrake.trajectories import PiecewisePolynomial

In [ ]:
# start a single meshcat server instance to use for remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])
web_url

## helper functions

In [ ]:
def visualize_transform(meshcat, name, transform, prefix='', length=0.15, radius=0.006):
    # Support RigidTransform as well as 4x4 homogeneous matrix.
    if isinstance(transform, RigidTransform):
        transform = transform.GetAsMatrix4()
    AddTriad(meshcat.vis, name=name, prefix=prefix, length=length, radius=0.005, opacity=0.2)
    meshcat.vis[prefix][name].set_transform(transform)

# used for getting the initial pose of the robot
def setup_manipulation_station(T_world_objectInitial):
    builder = DiagramBuilder()
    station = builder.AddSystem(ManipulationStation(time_step=1e-3))
    station.SetupClutterClearingStation()
    station.AddManipulandFromFile(
        #"drake/examples/manipulation_station/models/061_foam_brick.sdf",
        "drake/examples/manipulation_station/models/sphere.sdf",
        T_world_objectInitial)
    station.Finalize()

    frames_to_draw = {"gripper": {"body"}}
    meshcat = ConnectMeshcatVisualizer(builder,
          station.get_scene_graph(),
          output_port=station.GetOutputPort("pose_bundle"),
          delete_prefix_on_load=False,
          frames_to_draw=frames_to_draw,
          zmq_url=zmq_url)

    diagram = builder.Build()

    plant = station.get_multibody_plant()
    context = plant.CreateDefaultContext()
    gripper = plant.GetBodyByName("body")

    initial_pose = plant.EvalBodyPoseInWorld(context, gripper)

    simulator = Simulator(diagram)
    simulator.set_target_realtime_rate(1.0)
    simulator.AdvanceTo(0.01)
    
    return initial_pose, meshcat

# manipulation station has a lot of extra things that we don't need for IK
def CreateIiwaControllerPlant():
    """creates plant that includes only the robot and gripper, used for controllers."""
    robot_sdf_path = FindResourceOrThrow(
        "drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf")
    gripper_sdf_path = FindResourceOrThrow(
        "drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf")
    sim_timestep = 1e-3
    plant_robot = MultibodyPlant(sim_timestep)
    parser = Parser(plant=plant_robot)
    parser.AddModelFromFile(robot_sdf_path)
    parser.AddModelFromFile(gripper_sdf_path)
    plant_robot.WeldFrames(
        A=plant_robot.world_frame(),
        B=plant_robot.GetFrameByName("iiwa_link_0"))
    plant_robot.WeldFrames(
        A=plant_robot.GetFrameByName("iiwa_link_7"),
        B=plant_robot.GetFrameByName("body"),
        X_AB=RigidTransform(RollPitchYaw(np.pi/2, 0, np.pi/2), np.array([0, 0, 0.114]))
    )
    plant_robot.mutable_gravity_field().set_gravity_vector([0, 0, 0])
    plant_robot.Finalize()

    link_frame_indices = []
    for i in range(8):
        link_frame_indices.append(
            plant_robot.GetFrameByName("iiwa_link_" + str(i)).index())

    return plant_robot, link_frame_indices

def BuildAndSimulateTrajectory(q_traj, g_traj, T_world_objectInitial, T_world_target):
    """Simulate trajectory for manipulation station.
    @param q_traj: Trajectory class used to initialize TrajectorySource for joints.
    @param g_traj: Trajectory class used to initialize TrajectorySource for gripper.
    """
    builder = DiagramBuilder()
    station = builder.AddSystem(ManipulationStation(time_step=1e-4))
    station.SetupClutterClearingStation()
    station.AddManipulandFromFile(
        #"drake/examples/manipulation_station/models/061_foam_brick.sdf",
        "drake/examples/manipulation_station/models/sphere.sdf",
        T_world_objectInitial)
    station.Finalize()

    q_traj_system = builder.AddSystem(TrajectorySource(q_traj))
    g_traj_system = builder.AddSystem(TrajectorySource(g_traj))

    meshcat = ConnectMeshcatVisualizer(builder,
          station.get_scene_graph(),
          output_port=station.GetOutputPort("pose_bundle"),
          delete_prefix_on_load=True,
          frames_to_draw={"gripper":{"body"}},
          zmq_url=zmq_url)

    builder.Connect(q_traj_system.get_output_port(),
                  station.GetInputPort("iiwa_position"))
    builder.Connect(g_traj_system.get_output_port(),
                  station.GetInputPort("wsg_position"))

    diagram = builder.Build()

    simulator = Simulator(diagram)
    simulator.set_target_realtime_rate(1.0)
    station_plant = station.get_multibody_plant()

    return simulator, station_plant, meshcat

## create the trajectory

In [ ]:
"""
Throwing related trajectory stuff 
"""
def orientation_slerp(r_A, r_B):
    # assume t is between 0 and 1; responsibility of caller to scale time
    traj = PiecewiseQuaternionSlerp()
    traj.Append(0.0, r_A)
    traj.Append(1.0, r_B)
    return traj

def interpolatePosesLinear(T_world_poseA, T_world_poseB, t):
    # assume t is between 0 and 1; responsibility of caller to scale time
    p_A, r_A = T_world_poseA.translation(), T_world_poseA.rotation()
    p_B, r_B = T_world_poseB.translation(), T_world_poseB.rotation()
    
    # rotation is a clean slerp
    r_slerp = orientation_slerp(r_A, r_B)
    r_curr = RotationMatrix(r_slerp.value(t))
    
    # position is a straight line
    p_curr = p_A + t * (p_B - p_A)
    
    T_world_currGripper = RigidTransform(r_curr, p_curr)
    return T_world_currGripper

def interpolatePosesArcMotion(T_world_poseA, T_world_poseB, t):
    # assume t is between 0 and 1; responsibility of caller to scale time
    p_A, r_A = T_world_poseA.translation(), T_world_poseA.rotation()
    p_B, r_B = T_world_poseB.translation(), T_world_poseB.rotation()
    
    # rotation is a clean slerp
    r_slerp = orientation_slerp(r_A, r_B)
    r_curr = RotationMatrix(r_slerp.value(t))
    
    # position is an arc that isn't necessarily axis aligned
    arc_radius = p_B[2] - p_A[2]
    phi = np.arctan2(p_B[1] - p_A[1], p_B[0] - p_A[0]) #xy direction heading
    theta = (t - 1) * np.pi / 2 # -90 to 0 degrees
    p_curr = p_A + np.array([
        arc_radius * np.cos(theta) * np.cos(phi), 
        arc_radius * np.cos(theta) * np.sin(phi), 
        arc_radius * np.sin(theta) + arc_radius
    ])
    
    T_world_currGripper = RigidTransform(r_curr, p_curr)
    return T_world_currGripper

def interpolatePosesArcMotion_pdot(T_world_poseA, T_world_poseB, t):
    # assume t is between 0 and 1; responsibility of caller to scale time
    p_A, r_A = T_world_poseA.translation(), T_world_poseA.rotation()
    p_B, r_B = T_world_poseB.translation(), T_world_poseB.rotation()
    
    # position is an arc that isn't necessarily axis aligned
    arc_radius = p_B[2] - p_A[2]
    phi = np.arctan2(p_B[1] - p_A[1], p_B[0] - p_A[0]) #xy direction heading
    theta = (t - 1) * np.pi / 2 # -90 to 0 degrees
    pdot_curr = (np.pi / 2) * np.array([
      - arc_radius * np.sin(theta) * np.cos(phi), 
      - arc_radius * np.sin(theta) * np.sin(phi), 
        arc_radius * np.cos(theta)
    ])
    
    return pdot_curr

def get_launch_speed_required(theta, x, y, g=9.81):
    """
    Assuming we launch from (0, 0) at theta (radians),
    how fast do we need to launch to hit (x, y)?
    """
    assert 0 < theta < np.pi
    assert 0 < x
    assert y < np.tan(theta) * x
    
    time_to_target = np.sqrt(2.0 / g * (np.tan(theta) * x - y))
    speed_required = x / (np.cos(theta) * time_to_target)
    
    return speed_required

In [ ]:
def plan_throw(
    T_world_robotInitial,
    T_world_gripperObject,
    p_world_target, # np.array of shape (3,)
    gripper_to_object_dist,
    throw_height=0.5, # meters
    prethrow_height=0.2,
    prethrow_radius=0.4,
    throw_angle=np.pi/4.0,
    meshcat=None,
    throw_speed_adjustment_factor=1.0,
):
    """
    only works with the "back portion" of the clutter station until we figure out how to move the bins around 
    motion moves along an arc from a "pre throw" to a "throw" position
    """
    theta = 1.0 * np.arctan2(p_world_target[1], p_world_target[0])
    print(f"theta={theta}")
    
    T_world_prethrow = RigidTransform(
        p=np.array([
            prethrow_radius * np.cos(theta), 
            prethrow_radius * np.sin(theta), 
            prethrow_height
        ]),
        R=RotationMatrix.MakeXRotation(-np.pi/2).multiply(
            RotationMatrix.MakeYRotation((theta - np.pi / 2))
        )
    )
    
    throw_radius = throw_height - prethrow_height
    T_world_throw = RigidTransform(
        p=T_world_prethrow.translation() + np.array([
            throw_radius * np.cos(theta), 
            throw_radius * np.sin(theta), 
            throw_height - prethrow_height
        ]),
        R=RotationMatrix.MakeXRotation(-np.pi/2).multiply(
            RotationMatrix.MakeYRotation((theta - np.pi / 2)).multiply(
                RotationMatrix.MakeXRotation(-np.pi/2)
            )
        )
    )
    
    if meshcat:
        visualize_transform(meshcat, "T_world_prethrow", T_world_prethrow)
        visualize_transform(meshcat, "T_world_throw", T_world_throw)
    
    p_world_object_at_launch = interpolatePosesArcMotion(
        T_world_prethrow, T_world_throw,
        t=throw_angle / (np.pi / 2.)
    ).translation() + np.array([0, 0, -gripper_to_object_dist])
    pdot_world_launch = interpolatePosesArcMotion_pdot(
        T_world_prethrow, T_world_throw,
        t=throw_angle / (np.pi / 2.)
    )
    launch_speed_base = np.linalg.norm(pdot_world_launch)
    launch_speed_required = get_launch_speed_required(
        theta=throw_angle,
        x=np.linalg.norm(p_world_target[:2]) - np.linalg.norm(p_world_object_at_launch[:2]),
        y=p_world_target[2] - p_world_object_at_launch[2]
    )
    total_throw_time = launch_speed_base / launch_speed_required / throw_speed_adjustment_factor
    print(f"p_world_object_at_launch={p_world_object_at_launch}")
    print(f"target={p_world_target}")
    print(f"dx={np.linalg.norm(p_world_target[:2]) - np.linalg.norm(p_world_object_at_launch[:2])}")
    print(f"dy={p_world_target[2] - p_world_object_at_launch[2]}")
    print(f"pdot_world_launch={pdot_world_launch}")
    print(f"total_throw_time={total_throw_time}")
    
    T_world_hackyWayPoint = RigidTransform(
        p=[-.6, -0.0, 0.6],
        R=RotationMatrix.MakeXRotation(-np.pi/2.0), #R_WORLD_PRETHROW, #RotationMatrix.MakeXRotation(-np.pi/2.0), 
    )
        
    # event timings (not all are relevant to pose and gripper)
    # initial pose => prethrow => throw => yays
    t_goToObj = 1.0
    t_holdObj = 0.5
    t_goToPreobj = 1.0
    t_goToWaypoint = 1.0
    t_goToPrethrow = 1.0
    t_goToRelease = total_throw_time * throw_angle / (np.pi / 2.)
    t_goToThrowEnd = total_throw_time * (1 - throw_angle / (np.pi / 2.))
    t_throwEndHold = 3.0
    ts = np.array([
        t_goToObj, 
        t_holdObj, 
        t_goToPreobj, 
        t_goToWaypoint, 
        t_goToPrethrow,
        t_goToRelease,
        t_goToThrowEnd,
        t_throwEndHold
    ])
    cum_pose_ts = np.cumsum(ts)
    print(cum_pose_ts)
    
    # Create pose trajectory
    t_lst = np.linspace(0, cum_pose_ts[-1], 1000)
    pose_lst = []
    for t in t_lst:
        if t < cum_pose_ts[1]:
            pose = interpolatePosesLinear(T_world_robotInitial, T_world_gripperObject, min(t / ts[0], 1.0))
        elif t < cum_pose_ts[2]:
            pose = interpolatePosesLinear(T_world_gripperObject, T_world_robotInitial, (t - cum_pose_ts[1]) / ts[2])
        elif t < cum_pose_ts[3]:
            pose = interpolatePosesLinear(T_world_robotInitial, T_world_hackyWayPoint, (t - cum_pose_ts[2]) / ts[3])
        elif t <= cum_pose_ts[4]:
            pose = interpolatePosesLinear(T_world_hackyWayPoint, T_world_prethrow, (t - cum_pose_ts[3]) / ts[4])
        else:
            pose = interpolatePosesArcMotion(T_world_prethrow, T_world_throw, min((t - cum_pose_ts[4]) / (ts[5] + ts[6]), 1.0)) 
        pose_lst.append(pose)
        
    # Create gripper trajectory. 
    gripper_times_lst = np.array([
        0.,
        t_goToObj, 
        t_holdObj, 
        t_goToPreobj, 
        t_goToWaypoint, 
        t_goToPrethrow,
        t_goToRelease, 
        t_goToThrowEnd,
        t_throwEndHold
    ])
    gripper_cumulative_times_lst = np.cumsum(gripper_times_lst)
    GRIPPER_OPEN = 0.5
    GRIPPER_CLOSED = 0.0
    gripper_knots = np.array([
        GRIPPER_OPEN, 
        GRIPPER_OPEN, 
        GRIPPER_CLOSED, 
        GRIPPER_CLOSED,  
        GRIPPER_CLOSED, 
        GRIPPER_CLOSED, 
        GRIPPER_CLOSED, 
        GRIPPER_OPEN,
        GRIPPER_CLOSED
    ]).reshape(1,gripper_times_lst.shape[0])
    g_traj = PiecewisePolynomial.FirstOrderHold(gripper_cumulative_times_lst, gripper_knots)
        
    return t_lst, pose_lst, g_traj

# Get initial pose of the gripper by using default context of manip station.
P_WORLD_TARGET = np.array([-4, 0, 2])
GRIPPER_TO_OBJECT_DIST = 0.13 # meters
T_world_objectInitial = RigidTransform(
    p=[-.1, -.69, 1.04998503e-01],
    R=RotationMatrix.MakeZRotation(np.pi/2.0)
)
T_world_gripperObject = RigidTransform(
    p=T_world_objectInitial.translation() + np.array([0, 0, GRIPPER_TO_OBJECT_DIST]),
    R=RotationMatrix.MakeXRotation(-np.pi/2.0)
)
T_world_robotInitial, meshcat = setup_manipulation_station(T_world_objectInitial)

#object frame viz
visualize_transform(meshcat, "T_world_obj0", T_world_objectInitial)

t_lst, pose_lst, g_traj = plan_throw(
    T_world_robotInitial=T_world_robotInitial,
    T_world_gripperObject=T_world_gripperObject,
    p_world_target=P_WORLD_TARGET,
    gripper_to_object_dist=GRIPPER_TO_OBJECT_DIST,
    #meshcat=meshcat
)

#for t, pose in enumerate(pose_lst[:300]):
#    visualize_transform(meshcat, str(t), pose)

In [ ]:
def create_q_knots(pose_lst):
    """Convert end-effector pose list to joint position list using series of 
    InverseKinematics problems. Note that q is 9-dimensional because the last 2 dimensions 
    contain gripper joints, but these should not matter to the constraints.
    @param: pose_lst (python list): post_lst[i] contains keyframe X_WG at index i.
    @return: q_knots (python_list): q_knots[i] contains IK solution that will give f(q_knots[i]) \approx pose_lst[i].
    """
    q_knots = []
    plant, _ = CreateIiwaControllerPlant()
    world_frame = plant.world_frame()
    gripper_frame = plant.GetFrameByName("body")
    #q_nominal = np.array([ 0., 0.6, 0., -1.75, 0., 1., 0., 0., 0.]) # nominal joint for joint-centering.
    q_nominal = np.array([-1.57, 0.1, 0.00, -1.2, 0.00, 1.60, 0.00, 0.00, 0.00])

    def AddOrientationConstraint(ik, R_WG, bounds):
        """Add orientation constraint to the ik problem. Implements an inequality 
        constraint where the axis-angle difference between f_R(q) and R_WG must be
        within bounds. Can be translated to:
        ik.prog().AddBoundingBoxConstraint(angle_diff(f_R(q), R_WG), -bounds, bounds)
        """
        ik.AddOrientationConstraint(
            frameAbar=world_frame, R_AbarA=R_WG, 
            frameBbar=gripper_frame, R_BbarB=RotationMatrix(),
            theta_bound=bounds
        )

    def AddPositionConstraint(ik, p_WG_lower, p_WG_upper):
        """Add position constraint to the ik problem. Implements an inequality
        constraint where f_p(q) must lie between p_WG_lower and p_WG_upper. Can be
        translated to 
        ik.prog().AddBoundingBoxConstraint(f_p(q), p_WG_lower, p_WG_upper)
        """
        ik.AddPositionConstraint(
            frameA=world_frame, frameB=gripper_frame, p_BQ=np.zeros(3),
            p_AQ_lower=p_WG_lower, p_AQ_upper=p_WG_upper)

    print(f"NUM POSES: {len(pose_lst)}")
    for i in range(len(pose_lst)):
        if i % 100 == 0:
            print(i)
        ik = inverse_kinematics.InverseKinematics(plant)
        q_variables = ik.q() # Get variables for MathematicalProgram
        prog = ik.prog() # Get MathematicalProgram

        #### Modify here ###############################
        X_WG = pose_lst[i]
        p_WG = X_WG.translation()
        r_WG = X_WG.rotation()

        z_slack = 0
        degrees_slack = 0
        AddPositionConstraint(ik, p_WG - np.array([0, 0, z_slack]), p_WG + np.array([0, 0, z_slack]))
        AddOrientationConstraint(ik, r_WG, degrees_slack * np.pi / 180)

        # initial guess
        if i == 0:
            prog.SetInitialGuess(q_variables, q_nominal)
            diff = q_variables - q_nominal
        else:
            prog.SetInitialGuess(q_variables, q_knots[i - 1])
            diff = q_variables - q_knots[i - 1]

        prog.AddCost(np.sum(diff.dot(diff)))

        ################################################

        result = Solve(prog)
        if not result.is_success():
            visualize_transform(meshcat, "FAIL", X_WG, prefix='', length=0.3, radius=0.02)
            print(f"Failed at {i}")
            break
            #raise RuntimeError
        tmp = result.GetSolution(q_variables)
        q_knots.append(tmp)

    return q_knots

q_knots = np.array(create_q_knots(pose_lst))
q_traj = PiecewisePolynomial.CubicShapePreserving(t_lst, q_knots[:, 0:7].T)

In [ ]:
simulator, station_plant, meshcat = BuildAndSimulateTrajectory(q_traj, g_traj, T_world_objectInitial, None)
visualize_transform(
    meshcat,
    "TARGET",
    RigidTransform(RotationMatrix.MakeZRotation(0), P_WORLD_TARGET),
    prefix='',
    length=0.3,
    radius=0.02
)

meshcat.start_recording()
print(f"Running for {q_traj.end_time()} seconds")
simulator.AdvanceTo(q_traj.end_time())
#simulator.AdvanceTo(4.57016674)
meshcat.stop_recording()
meshcat.publish_recording()

In [ ]:
context = simulator.get_context()
station_plant.GetPositions(
    station_plant.GetMyContextFromRoot(context),
    station_plant.GetModelInstanceByName("sphere")
)

In [ ]:
[-6.12132034e-01  7.49645537e-17  1.57867966e-01]

In [ ]:
station_plant.gravity_field().gravity_vector()